In [69]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf


*Collecting Data from Yfinance*

In [70]:

start='2012-01-01'
end='2022-12-21'

stock='GOOG'

data=yf.download(stock, start, end)

[*********************100%***********************]  1 of 1 completed


*Replacing Index from Dates to Numbers*

In [71]:

data.reset_index(inplace=True)

In [72]:
data 

,Date,Open,High,Low,Close,Adj Close,Volume
0,2012-01-03,16.262545,16.641375,16.248346,16.573130,16.532528,147611217
1,2012-01-04,16.563665,16.693678,16.453827,16.644611,16.603836,114989399
2,2012-01-05,16.491436,16.537264,16.344486,16.413727,16.373516,131808205
3,2012-01-06,16.417213,16.438385,16.184088,16.189817,16.150156,108119746
4,2012-01-09,16.102144,16.114599,15.472754,15.503389,15.465409,233776981
...,...,...,...,...,...,...,...
2756,2022-12-14,95.540001,97.220001,93.940002,95.309998,95.076508,26452900
2757,2022-12-15,93.540001,94.029999,90.430000,91.199997,90.976578,28298800
2758,2022-12-16,91.199997,91.750000,90.010002,90.860001,90.637405,48485500
2759,2022-12-19,90.879997,91.199997,88.925003,89.150002,88.931602,23020500


*Creating 100 days and 200 days days moving average of close price*

In [73]:
ma_100_days=data.Close.rolling(100).mean()
ma_200_days=data.Close.rolling(200).mean()


*If there are any null values the data will be deleted*

In [74]:
data.dropna(inplace=True)

*Splitting the data into training and test set*

In [75]:
data_train = pd.DataFrame(data.Close[0:int(len(data)*0.8)])
data_test = pd.DataFrame(data.Close[int(len(data)*0.8):len(data)])

*Defining a function which will data between 0 and 1*

In [76]:
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

*Fitting the Training Data*

In [77]:
data_train_scale=scaler.fit_transform(data_train)


Array Slicing

In [78]:
x=[]
y=[]
for i in range(100,data_train_scale.shape[0]):
    x.append(data_train_scale[i-100:i])
    y.append(data_train_scale[i,0])

*Changing the list into an array for both X and Y*

In [79]:
x,y=np.array(x), np.array(y)

*Importing Keras Libraries*

In [80]:
from keras.layers import Dense, Dropout, LSTM
from keras.models import Sequential

*Creating model Sequential and training it with LSTM layers where the mentioned units are the dimensions and the return _sequences means the output of first is the input for the second and defining the input shape*

In [81]:
model=Sequential()
model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=((x.shape[1],1))))
model.add(Dropout(0.2))
model.add(LSTM(units=60, activation='relu', return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(units=80, activation='relu', return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(units=120, activation='relu'))
model.add(Dropout(0.5))


model.add(Dense(units=1))


*Compiling the model*

In [82]:
model.compile(optimizer='adam', loss='mean_squared_error')

*Fitting the model*

In [83]:
model.fit(x,y, epochs=70, batch_size=32, verbose=1)

Epoch 1/70
66/66 [==============================] - 13s 138ms/step - loss: 0.0304
Epoch 2/70
66/66 [==============================] - 10s 145ms/step - loss: 0.0074
Epoch 3/70
66/66 [==============================] - 10s 145ms/step - loss: 0.0055
Epoch 4/70
66/66 [==============================] - 9s 142ms/step - loss: 0.0051
Epoch 5/70
66/66 [==============================] - 10s 155ms/step - loss: 0.0046
Epoch 6/70
66/66 [==============================] - 9s 140ms/step - loss: 0.0052
Epoch 7/70
66/66 [==============================] - 9s 139ms/step - loss: 0.0047
Epoch 8/70
66/66 [==============================] - 9s 136ms/step - loss: 0.0041
Epoch 9/70
66/66 [==============================] - 9s 138ms/step - loss: 0.0036
Epoch 10/70
66/66 [==============================] - 9s 137ms/step - loss: 0.0036
Epoch 11/70
66/66 [==============================] - 9s 136ms/step - loss: 0.0036
Epoch 12/70
66/66 [==============================] - 9s 138ms/step - loss: 0.0033
Epoch 13/70
66/66 [==

*Collecting the last 100 days data of training in a variable*

In [84]:
pas_100_days=data_train.tail(100)

*Concatinating the last 100 days data and the test data*

In [85]:
data_test=pd.concat([pas_100_days,data_test], ignore_index=True)
print(len(data_test))

653


*Scaling the test data between 0 and 1*

In [86]:
data_test_scale= scaler.fit_transform(data_test)


*Array Slicing*

In [87]:
x=[]
y=[]
for i in range(100,data_test_scale.shape[0]):
    x.append(data_test_scale[i-100:i])
    y.append(data_test_scale[i,0])
x,y=np.array(x), np.array(y)


*Predicting the values of X*

In [88]:

y_predict=model.predict(x)

18/18 [==============================] - 1s 40ms/step


*Descaling the vpredicted values and the original values for plotting of graph*

In [89]:
descale =1/scaler.scale_
y_predict= y_predict*descale
y=y*descale

*Saving this model in a keras file so it can be used by Streamlit for web*

In [91]:
model.save("Stock_Prediction_Model.keras")